In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 10      # number of variables
num_ineq = 10     # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msLinear
model = msLinear(num_var, num_ineq, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_exact_10-10.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:17<00:00,  5.86it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    -6.543142          0.156450      0.168937
std      2.740046          0.724135      0.016477
min    -10.685309          0.000000      0.122423
25%     -8.805172          0.000000      0.155694
50%     -7.096373          0.000000      0.169717
75%     -4.556288          0.000000      0.183881
max     -0.486817          4.446263      0.227157
Number of infeasible solution: 5


## Heuristic - Round

In [8]:
from src.heuristic import naive_round

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("gurobi")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_heur_rnd_10-10.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.22it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    -6.706395          0.303520      0.128860
std      2.714646          0.662586      0.006523
min    -10.761161          0.039067      0.115122
25%     -9.129486          0.091426      0.125674
50%     -7.227390          0.149238      0.129468
75%     -4.554637          0.262538      0.132903
max     -1.136514          4.600931      0.145252
Number of infeasible solution: 100


## Heuristic - N1

In [10]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_heur_n1_10-10.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.40it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    -2.737939          0.109072      0.133490
std      3.675763          0.489871      0.010326
min    -10.031817          0.000000      0.120450
25%     -4.997535          0.000000      0.123700
50%     -2.525564          0.000000      0.136216
75%     -0.051590          0.000000      0.139036
max      4.053926          2.728475      0.169357
Number of infeasible solution: 5


## Learnable Rounding

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 32            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmLinear
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmLinear(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 248.32
Epoch 1, Validation Loss: 83.78
Epoch 2, Validation Loss: 34.73
Epoch 3, Validation Loss: 31.79
Epoch 4, Validation Loss: 29.40
Epoch 5, Validation Loss: 21.80
Epoch 6, Validation Loss: 22.70
Epoch 7, Validation Loss: 16.79
Epoch 8, Validation Loss: 16.22
Epoch 9, Validation Loss: 14.56
Epoch 10, Validation Loss: 14.13
Epoch 11, Validation Loss: 11.27
Epoch 12, Validation Loss: 9.56
Epoch 13, Validation Loss: 10.61
Epoch 14, Validation Loss: 11.23
Epoch 15, Validation Loss: 10.18
Epoch 16, Validation Loss: 11.89
Epoch 17, Validation Loss: 7.42
Epoch 18, Validation Loss: 8.62
Epoch 19, Validation Loss: 11.07
Epoch 20, Validation Loss: 6.21
Epoch 21, Validation Loss: 5.44
Epoch 22, Validation Loss: 6.71
Epoch 23, Validation Loss: 16.72
Epoch 24, Validation Loss: 6.04
Epoch 25, Validation Loss: 13.17
Epoch 26, Validation Loss: 5.53
Epoch 27, Validation Loss: 6.25
Epoch 28, Validation Loss: 6.31
Epoch 29, Validation Loss: 10.74
Epoch 30, Validation Loss: 7.

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_lr_10-10.csv")

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 221.71it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    -3.534988          0.051967      0.002836
std      2.269858          0.141607      0.000779
min     -8.342488          0.000000      0.001433
25%     -5.412907          0.000000      0.002116
50%     -3.093637          0.000000      0.002695
75%     -1.410679          0.001665      0.003336
max     -0.135913          0.807881      0.005119
Number of infeasible solution: 25


## Learnable Threshold

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 32            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [19]:
# set problem
import neuromancer as nm
from src.problem import nmLinear
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmLinear(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [20]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 253.56
Epoch 1, Validation Loss: 89.64
Epoch 2, Validation Loss: 36.35
Epoch 3, Validation Loss: 25.39
Epoch 4, Validation Loss: 24.80
Epoch 5, Validation Loss: 18.64
Epoch 6, Validation Loss: 15.05
Epoch 7, Validation Loss: 15.70
Epoch 8, Validation Loss: 14.80
Epoch 9, Validation Loss: 15.31
Epoch 10, Validation Loss: 11.47
Epoch 11, Validation Loss: 11.77
Epoch 12, Validation Loss: 10.27
Epoch 13, Validation Loss: 10.60
Epoch 14, Validation Loss: 11.09
Epoch 15, Validation Loss: 12.05
Epoch 16, Validation Loss: 9.06
Epoch 17, Validation Loss: 12.05
Epoch 18, Validation Loss: 10.70
Epoch 19, Validation Loss: 11.96
Epoch 20, Validation Loss: 8.56
Epoch 21, Validation Loss: 8.11
Epoch 22, Validation Loss: 8.20
Epoch 23, Validation Loss: 7.31
Epoch 24, Validation Loss: 7.81
Epoch 25, Validation Loss: 9.03
Epoch 26, Validation Loss: 7.64
Epoch 27, Validation Loss: 7.05
Epoch 28, Validation Loss: 6.00
Epoch 29, Validation Loss: 6.18
Epoch 30, Validation Loss: 8.5

In [21]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_lt_10-10.csv")

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 199.28it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    -1.243678          0.053696      0.003104
std      0.201618          0.146035      0.000749
min     -1.750116          0.000000      0.001997
25%     -1.384379          0.000000      0.002512
50%     -1.252345          0.000000      0.003005
75%     -1.102529          0.000000      0.003524
max     -0.721665          0.813297      0.004787
Number of infeasible solution: 24


### Parametric Learning Then Rounding

In [22]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [23]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 32            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [24]:
# set problem
import neuromancer as nm
from src.problem import nmLinear
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmLinear(["b", "x"], num_var, num_ineq, penalty_weight)

In [25]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 258.35
Epoch 1, Validation Loss: 96.16
Epoch 2, Validation Loss: 48.61
Epoch 3, Validation Loss: 30.55
Epoch 4, Validation Loss: 27.70
Epoch 5, Validation Loss: 25.67
Epoch 6, Validation Loss: 22.11
Epoch 7, Validation Loss: 20.14
Epoch 8, Validation Loss: 14.49
Epoch 9, Validation Loss: 11.83
Epoch 10, Validation Loss: 10.43
Epoch 11, Validation Loss: 9.57
Epoch 12, Validation Loss: 9.06
Epoch 13, Validation Loss: 10.60
Epoch 14, Validation Loss: 8.93
Epoch 15, Validation Loss: 8.17
Epoch 16, Validation Loss: 8.58
Epoch 17, Validation Loss: 7.54
Epoch 18, Validation Loss: 7.40
Epoch 19, Validation Loss: 7.35
Epoch 20, Validation Loss: 6.75
Epoch 21, Validation Loss: 5.63
Epoch 22, Validation Loss: 5.18
Epoch 23, Validation Loss: 4.33
Epoch 24, Validation Loss: 4.95
Epoch 25, Validation Loss: 3.85
Epoch 26, Validation Loss: 3.70
Epoch 27, Validation Loss: 3.95
Epoch 28, Validation Loss: 3.59
Epoch 29, Validation Loss: 3.31
Epoch 30, Validation Loss: 4.78
Epoch

In [26]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_pr_10-10.csv")

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 342.34it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    -3.697510          0.101636      0.000801
std      2.434732          0.156273      0.000503
min     -8.440810          0.000000      0.000000
25%     -5.345812          0.008144      0.000379
50%     -3.428675          0.051423      0.001000
75%     -1.084636          0.105345      0.001010
max     -0.717444          0.879817      0.002259
Number of infeasible solution: 81
